In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [0]:
# from pyspark.sql import SparkSession  -> not necessary in databricks

In [0]:
# spark = SparkSession.builder.getOrCreate()

In [0]:
# 1. method

data_rows = [
    (100, 'Sandeep', 20000),
    (200, 'Jashu', 14000),
    (300, 'Ramya', 30000),
    (400, 'Charan', 20000),
    (500, 'Dileep', 25000),
    (600, 'Reddy', 15000)
]

columns = ["emp_id", "emp_name", "salary"]

emp_df = spark.createDataFrame(data=data_rows, schema=columns)

emp_df.show()

In [0]:
emp_df.rdd.getNumPartitions()

In [0]:
display(emp_df)

Databricks data profile. Run in Databricks to view.

In [0]:
# 2. method

data_rows = [
    (100, 'Sandeep', 20000),
    (200, 'Jashu', 14000),
    (300, 'Ramya', 30000),
    (400, 'Charan', 20000),
    (500, 'Dileep', 25000),
    (600, 'Reddy', 15000),
    (100, 'Sandeep', 20000),
    (200, 'Jashu', 24000),
    (700, 'Dileep', None),
    (800, 'Reddy', None),
]

schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("emp_name", StringType(), True),
    StructField("salary", IntegerType(), True),
])

emp_df2 = spark.createDataFrame(data=data_rows, schema=schema)
emp_df2.display()

In [0]:
# challenge - 1 --> create a new column called status and assign a default bool value (true)

In [0]:
emp_df = (
    emp_df2
    .withColumn("status", F.lit(True))
)
emp_df.display()

In [0]:
# challenge 2 --> rename salary column to emp_salary

emp_df = (
    emp_df
    .withColumnRenamed("salary", "emp_salary")
)
emp_df.display()

In [0]:
# 4. How to derive multiple columns
emp_df3 = (
    emp_df2
    .withColumn("status", F.lit(True))
    .withColumn("gender", F.lit("Male"))
    .withColumn("department", F.lit("IT"))
    .withColumn("effective_date", F.lit(F.current_date()))
)

emp_df3.display()

In [0]:
# in spark 3.5 version - we have new update --> withColumns

emp_df4 = (
    emp_df2
    .withColumns({
        "status": F.lit(True),
        "gender": F.lit("Male"),
        "department": F.lit("IT"),
        "effective_date": F.lit(F.current_date())
    })
)

emp_df4.display()

In [0]:
# selecting the columns from dataframe
# 1st way
display(emp_df4.select("emp_id", "emp_name"))

In [0]:
# 2nd method
display(emp_df4.select(emp_df4.emp_id, emp_df4.emp_name))

In [0]:
# 3rd method
display(emp_df4.select(emp_df4["emp_id"], emp_df4["emp_name"]))

In [0]:
# 4th method
display(emp_df4.select(F.col("emp_id"), F.col("emp_name")))

In [0]:
# filter | where

emp_df5 = emp_df4.filter(F.col("salary") > 20000)
emp_df5.display()

In [0]:
emp_df5 = emp_df4.where(F.col("salary") > 20000)
emp_df5.display()

In [0]:
# conditional logic derivation in withColumn

emp_df4 = (
    emp_df4
    .withColumn(
        "is_highly_paid",
        F.when(
            F.col("salary") > 20000, True
        )
        .otherwise(False)
    )
)
display(emp_df4)

In [0]:
# drop columns from dataframe 

emp_df4 = emp_df4.drop("gender")
display(emp_df4)

In [0]:
# distinct
display(emp_df4.distinct())

In [0]:
# dropDuplicates

emp_df4 = emp_df4.dropDuplicates(subset=["emp_id"])
display(emp_df4)

In [0]:
# sortBy and orderBy

display(emp_df4.sort(F.col("emp_id").desc()))

In [0]:
# sortBy and orderBy

display(emp_df4.orderBy(F.col("emp_id").desc()))

In [0]:
display(
    emp_df.select(
        F.col("status").alias("is_active")
    )
)

Group By | Agg

In [0]:
emp_df4.display()

In [0]:
emp_df5 = (
    emp_df4
    .groupBy("is_highly_paid")
    .count()
    # .max("salary")
)
emp_df5.display()

In [0]:
emp_df5 = (
    emp_df4
    .groupBy("is_highly_paid")
    .agg(
        F.count("is_highly_paid").alias("noOfEmp"),
        F.max("salary").alias("top_salary")
    )
)
emp_df5.display()

In [0]:
# filter null values only

display(emp_df4.filter(F.col("salary").isNull()))

In [0]:
min_salary = 10_000

emp_df6 = (
    emp_df4
    .fillna(min_salary)
)
display(emp_df6)

In [0]:
emp_df7 = (
    emp_df4
    .fillna({"salary": min_salary})
)
display(emp_df7)

In [0]:
emp_df8 = (
    emp_df4
    .na.fill({"salary": min_salary})
)
display(emp_df8)

In [0]:
# example write